In [5]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings

In [1]:
label_dict = {0: '긍정', 1: '부정', 2: '중립'}

In [2]:
from transformers import TextClassificationPipeline, BertTokenizerFast, TFBertForSequenceClassification, BertTokenizer
import os


# MODEL_NAME = 'fine-tuned-klue-bert-base'
# MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

MODEL_SAVE_PATH = '/home/inter/chat_MBTI/NP_classifier'

# Load Fine-tuning model
# tokenizer = BertTokenizer.from_pretrained("klue/bert-base")
tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    framework='tf',
    top_k=None
)

2025-11-16 22:08:27.336595: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 22:08:27.390381: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-16 22:08:29.359213: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1763298512.647635    1828 gpu_device.cc:2020] Created device /job:localhost/rep

In [3]:
max_seq_len = 100
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, padding='max_length')
    # print('input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True) :',input_id)

    padding_count = input_id.count(tokenizer.pad_token_id)
    # print('padding_count = input_id.count(tokenizer.pad_token_id) :',padding_count)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    # print('attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count :',attention_mask)
    token_type_id = [0] * max_seq_len
    # print('token_type_id = [0] * max_seq_len :',token_type_id)

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]
    # print('encoded_input = [input_ids, attention_masks, token_type_ids] :',encoded_input)

    score = np.argmax(model.predict(encoded_input)[0])
    # print('model.predict(encoded_input) :',model.predict(encoded_input))
    # print('model.predict(encoded_input)[0] :',model.predict(encoded_input)[0])
    # print('score = np.argmax(model.predict(encoded_input)[0]) :',score)

    # print('sentence :',new_sentence)
    print(label_dict[score])

In [6]:
while True :
  new_sentence = input('sentence > ')
  if new_sentence == '끝':
    break
  sentiment_predict(new_sentence)

1/1 [==============================] - 4s 4s/step
중립
1/1 [==============================] - 0s 121ms/step
부정
1/1 [==============================] - 0s 213ms/step
긍정
1/1 [==============================] - 0s 167ms/step
중립


In [7]:
import numpy as np

def NP_predict(sentence):

  temp_li = []
  score_li = text_classifier(sentence)[0]
  for i in score_li:
    temp_li.append(i['score'])

  return label_dict[int(score_li[temp_li.index(max(temp_li))]['label'][6:])]

In [9]:
NP_predict('아낌없이 나눠주세요')

'중립'